# 07 | Portfolio Story

## Executive Narrative
- **Problem**: Build a recruiter-ready, local-first lakehouse with real data and production engineering patterns.
- **Approach**: Notebook-first medallion design with deterministic orchestration, dbt marts, and quality checkpoints.
- **Outcome**: Reproducible Bronze/Silver/Gold pipeline with observable metrics and CI execution.

## Design Decisions
1. DuckDB + dbt-duckdb for zero-infra analytics engineering.
2. Public dataset ingestion at runtime to avoid checked-in raw data.
3. Great Expectations checkpoint on Silver to gate downstream marts.
4. Papermill notebook orchestration for CI-friendly execution.

In [ ]:
# Parameters
source = "fivethirtyeight"
dataset = "recent_grads,bechdel_movies"
run_date = "2026-02-22"
force_refresh = False

In [ ]:
import sys
from pathlib import Path

ROOT_DIR = Path.cwd()
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

In [ ]:
import pandas as pd

from src.common.io import write_json
from src.common.paths import GOLD_DIR, REPORTS_DIR

kpi_major = pd.read_parquet(GOLD_DIR / 'kpi_major_outcomes' / 'data.parquet')
kpi_movie = pd.read_parquet(GOLD_DIR / 'kpi_movie_representation' / 'data.parquet')

portfolio_summary = {
    'total_majors': int(kpi_major.loc[0, 'total_majors']),
    'avg_median_salary': float(kpi_major.loc[0, 'avg_median_salary']),
    'avg_unemployment_rate': float(kpi_major.loc[0, 'avg_unemployment_rate']),
    'movie_pass_rate': float(kpi_movie.loc[0, 'pass_rate']),
    'latest_movie_release_date': str(kpi_movie.loc[0, 'latest_release_date']),
}

write_json(REPORTS_DIR / 'portfolio_summary.json', portfolio_summary)

pd.DataFrame([portfolio_summary])